Importamos librerias

In [381]:
import numpy as np
import pandas as pd


In [382]:
import sys
try:
    import cplex
except:
    if hasattr(sys, 'real_prefix'):
        #we are in a virtual env.
        !pip install cplex
    else:
        !pip install --user cplex

In [383]:
import sys
try:
    import docplex.mp
except:
    if hasattr(sys, 'real_prefix'):
        #we are in a virtual env.
        !pip install docplex
    else:
        !pip install --user docplex

In [384]:
import docplex.cp
from docplex.cp.model import *


Visualizamos los dataframes con los parametros y los productos

In [385]:
dfproductos = pd.read_csv("productos.csv")

In [386]:
dfproductos

,Unnamed: 0,idprod,stockInicial,demandaMensual,precioUSD,pedidoMinimo,provedorID
0,0,1.0,371.0,39.0,615.0,553.0,4.0
1,0,2.0,324.0,39.0,775.0,282.0,3.0
2,0,3.0,372.0,34.0,738.0,458.0,2.0
3,0,4.0,285.0,34.0,735.0,328.0,2.0
4,0,5.0,398.0,39.0,360.0,830.0,4.0
5,0,6.0,386.0,33.0,447.0,230.0,4.0
6,0,7.0,246.0,33.0,252.0,732.0,3.0
7,0,8.0,274.0,34.0,408.0,342.0,4.0
8,0,9.0,340.0,46.0,166.0,408.0,1.0
9,0,10.0,322.0,50.0,33.0,731.0,4.0


In [387]:
dfproductos["demandaMensual"][1]

39.0

In [388]:
dfProvedores = pd.read_csv("provedores.csv")

In [389]:
dfProvedores

,Unnamed: 0,provedorID,minimoMontoPedido,puertoEnvio
0,0,1.0,10313.0,1.0
1,0,2.0,10410.0,3.0
2,0,3.0,8884.0,2.0
3,0,4.0,9197.0,1.0


In [390]:
dfGeneral = pd.read_csv("parametrosGenerales.csv")

In [391]:
dfGeneral

,Unnamed: 0,interesPlataMuerta,meses,productos,provedores,puertos,costoDespachante,delayEntrega
0,0,0.025,8,12,4,3,1000,3


In [392]:
dfGeneral["interesPlataMuerta"][0]

0.025

Nos creamos el conjunto para iterar de cada tipo

In [393]:
Meses = list(range(1,int(dfGeneral["meses"] + 1)))
Prods = list(range(1,int(dfGeneral["productos"] + 1)))
Provs = list(range(1,int(dfGeneral["provedores"] + 1)))
Puertos = list(range(1,int(dfGeneral["puertos"] + 1)))

In [394]:
Meses

[1, 2, 3, 4, 5, 6, 7, 8]

Definimos el modelo

In [395]:
mdl = CpoModel(name="modeladoImportacion")

Definimos las variables de costos acumuladas

In [396]:
varCostosAcumulados = {}
varAduanas = {}
varPenalizacionPlataMuerta = {}
for m in Meses:
    varCostosAcumulados[m] = integer_var(name="costosAcum_m".format(m))
    varAduanas[m] = integer_var(name="aduanas_m".format(m))
    varPenalizacionPlataMuerta[m] = integer_var(name="penalPlata_m".format(m))

In [397]:
mdl.add(varCostosAcumulados[1] == 0) #  El costo inicial el primer mes es 0


Definimos la funcion objetivo como la minimizacion de los costos acumulados hasta el ultimo mes

In [398]:
mdl.minimize(varCostosAcumulados[len(Meses)])

Definicion a traves de restriccion de que es el costo acumulado

In [399]:
for m in Meses:
    if(m>=2):
        mdl.add(
            varCostosAcumulados[m] >= 
                varCostosAcumulados[m-1] + 
                varAduanas[m-1]*dfGeneral["costoDespachante"][0] +
                varPenalizacionPlataMuerta[m-1]*dfGeneral["interesPlataMuerta"][0]) 


In [400]:
varStockProdMes = {}
for m in Meses:
    for i in Prods:
        varStockProdMes[(i, m)] = integer_var(name="stock_produc_mes".format(i, m))

In [401]:
for m in Meses:
    suma = 0
    for i in Prods:
        suma = suma + (varStockProdMes[(i, m)]- dfproductos["demandaMensual"][i-1]) * dfproductos["precioUSD"][i-1]
    mdl.add(
        varPenalizacionPlataMuerta[m] >= suma
               )

In [402]:
# Esta restriccion puede que sea redundante
for m in Meses:
    for i in Prods:
       mdl.add(varStockProdMes[(i, m)] >= dfproductos["demandaMensual"][i-1])

In [403]:
deltaProdProvMes = {}
cantProdProvMes = {}

for m in Meses:
    for i in Prods:
        for p in Provs:
            deltaProdProvMes[(i, p,m)] = binary_var(name="delta_prov_produc_mes".format(i, p, m))
            cantProdProvMes[(i, p,m)] = integer_var(name="cant_prov_produc_mes".format(i, p, m))

In [404]:
thetaProvMes = {}

for m in Meses:
        for p in Provs:
            thetaProvMes[( p,m)] = binary_var(name="theta_prov__mes".format( p, m))

In [405]:
for m in Meses:
    for i in Prods:
        for p in Provs:
            mdl.add(deltaProdProvMes[(i, p,m)] * dfproductos["pedidoMinimo"][i-1] <= cantProdProvMes[(i, p,m)])

In [406]:
M = 1000000 # NUNCA SE PIDE MAS DE 1 MILLON DE UN PRODUCTO, y si es que pedis la indicadora es 1
for m in Meses:
    for i in Prods:
        for p in Provs:
            mdl.add(deltaProdProvMes[(i, p,m)] * M >= cantProdProvMes[(i, p,m)])

In [407]:
for m in Meses:
    for p in Provs:
        sumaIndicadoras = 0
        for i in Prods:
            sumaIndicadoras = sumaIndicadoras + deltaProdProvMes[(i, p,m)]
        mdl.add(sumaIndicadoras <= M *  thetaProvMes[( p,m)])

In [408]:
for m in Meses:
    for p in Provs:
        sumaIndicadoras2 = 0
        for i in Prods:
            sumaIndicadoras2 = sumaIndicadoras2 + deltaProdProvMes[(i, p,m)]
        mdl.add(sumaIndicadoras2 >=  thetaProvMes[( p,m)])

In [409]:
for m in Meses:
    for i in Prods:
        for p in Provs:
            mdl.add(1 - (p - dfproductos["provedorID"][i-1] )/10000 >=  deltaProdProvMes[(i, p,m)])
            mdl.add(1 - (dfproductos["provedorID"][i-1] - p)/10000 >=  deltaProdProvMes[(i, p,m)])

In [410]:
for m in Meses:
    for p in Provs:
        sumaAcumulado = 0
        for i in Prods:
            sumaAcumulado = sumaAcumulado + cantProdProvMes[(i, p,m)]*dfproductos["precioUSD"][i-1]
        mdl.add(sumaAcumulado >= dfProvedores["minimoMontoPedido"][p-1] *  thetaProvMes[( p,m)])

In [411]:
for i in Prods:
    mdl.add(varStockProdMes[(i, 1)] == dfproductos["stockInicial"][i-1])

In [412]:
delay = dfGeneral["delayEntrega"][0]
for m in range(delay+1, Meses[len(Meses)-1] + 1):
    for i in Prods:
        sumaCantidad = 0
        for p in Provs:
            sumaCantidad = sumaCantidad + cantProdProvMes[(i, p,m - delay)]
            
        mdl.add(varStockProdMes[(i, m)] == varStockProdMes[(i, m-1)] - 
                                            dfproductos["demandaMensual"][i-1] + 
                                               sumaCantidad )

In [413]:
for m in range(2, delay+1):
    for i in Prods:
        mdl.add(varStockProdMes[(i, m)] == varStockProdMes[(i, m-1)] - dfproductos["demandaMensual"][i-1])

In [414]:
puertPuertoMes = {}

for m in Meses:
        for k in Puertos:
            puertPuertoMes[( k,m)] = binary_var(name="puerto__mes".format( k, m))

In [415]:
for m in Meses:
    for k in Puertos:
        for p in Provs:
            if(dfProvedores["puertoEnvio"][p-1] == k):
                mdl.add(puertPuertoMes[( k,m)] >= thetaProvMes[( p,m)])

In [416]:
for m in Meses:
    sumaPuertos = 0
    for k in Puertos:
        sumaPuertos = sumaPuertos + puertPuertoMes[( k,m)]
    mdl.add(varAduanas[m] >= sumaPuertos)

In [417]:
msol = mdl.solve()

 ! --------------------------------------------------- CP Optimizer 22.1.0.0 --
 ! Minimization problem - 944 variables, 1880 constraints
 ! Presolve      : 104 extractables eliminated
 ! Initial process time : 0.03s (0.03s extraction + 0.00s propagation)
 !  . Log search space  : 4324.0 (before), 4324.0 (after)
 !  . Memory usage      : 841.3 kB (before), 841.3 kB (after)
 ! Using parallel search with 4 workers.
 ! ----------------------------------------------------------------------------
 !          Best Branches  Non-fixed    W       Branch decision
                        0        944                 -
 + New bound is 171342
                        0        319    1            -
 + New bound is 174340
                        0        319    1   F        -
 + New bound is 174341
 *        225716      137  0.17s        1      (gap is 22.76%)
 *        193171      672  0.17s        1      (gap is 9.75%)
          193171     1000          1    1            -
 *        180136      141

In [418]:
msol.get_value(varCostosAcumulados[4])

103242

In [419]:
msol.get_value(varStockProdMes[(1,4)])

254

In [420]:
msol.get_value(varPenalizacionPlataMuerta[1])

1575323

In [421]:
msol.get_value(varPenalizacionPlataMuerta[4])

978941

In [422]:
msol.get_value(puertPuertoMes[1,4])

1

In [423]:

stock2 = []
for m in Meses:
    stock2.append(msol.get_value(varStockProdMes[(6,m)]))
stock2

[386, 353, 320, 287, 254, 221, 188, 155]

In [424]:
cantPedidos = 0
for m in Meses:
    for p in Provs:
        if(msol.get_value(thetaProvMes[( p,m)])==1):
            print("en mes "+ str(m) + "al prov " + str(p))
        cantPedidos = cantPedidos + msol.get_value(thetaProvMes[( p,m)])

en mes 4al prov 4
en mes 5al prov 1
en mes 5al prov 3
